In [10]:
import os
import math


In [12]:

def read_tsp(file_path):
    """Parses a .tsp file and extracts points as a list of (id, x, y)."""
    # Ensure the file path is correct relative to this script's directory
    # full_path = os.path.join(os.path.dirname(__file__), "..", file_path)
    points = []

    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()

        # Debugging: Print all lines for verification
        # print("DEBUG: File lines:\n", lines)

        # Skip the first 5 lines and process the coordinate section
        for line in lines[5:]:
            line = line.strip()

            if line == "EOF":  # Stop parsing at "EOF"
                break

            parts = line.split()
            if len(parts) == 3:  # Ensure it has id, x, y
                node_id, x, y = int(parts[0]), float(parts[1]), float(parts[2])
                points.append((node_id, x, y))
    except FileNotFoundError:
        raise FileNotFoundError(f"Input file not found: {full_path}")
    except Exception as e:
        raise RuntimeError(f"Error reading TSP file {full_path}: {e}")

    # Debugging: Print the parsed points for verification
    # print(f"DEBUG: Parsed points: {points}")

    return points

In [7]:


def create_distance_matrix(points):
    """Creates a distance matrix from points."""
    n = len(points)
    matrix = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(i + 1, n):
            dist = euclidean_distance(points[i], points[j])
            matrix[i][j] = matrix[j][i] = dist
    return matrix


def euclidean_distance(p1, p2):
    """Calculates the Euclidean distance between two points."""
    return math.sqrt((p1[1] - p2[1]) ** 2 + (p1[2] - p2[2]) ** 2)

In [9]:
read_tsp(file_path="../input/Atlanta.tsp")

[(1, 33665568.0, -84411070.0),
 (2, 33764940.0, -84371819.0),
 (3, 33770889.0, -84358622.0),
 (4, 34233391.0, -84103278.0),
 (5, 33921088.0, -84337342.0),
 (6, 34087634.0, -84530646.0),
 (7, 33683084.0, -84442307.0),
 (8, 33968483.0, -84204487.0),
 (9, 33809946.0, -84438909.0),
 (10, 34070898.0, -84277636.0),
 (11, 33903368.0, -84376924.0),
 (12, 33836703.0, -84481173.0),
 (13, 33792891.0, -84396721.0),
 (14, 33975761.0, -84265625.0),
 (15, 34002830.0, -84146049.0),
 (16, 34091985.0, -84275997.0),
 (17, 33788510.0, -84369260.0),
 (18, 33872130.0, -84261650.0),
 (19, 34003208.0, -84040883.0),
 (20, 33936017.0, -84388948.0)]

In [ ]:
from itertools import permutations
import time
from utils import create_distance_matrix


def brute_force_tsp(points, cutoff):
    """
    Finds the TSP tour using brute force within the time cutoff.
    :param points: List of (id, x, y) tuples.
    :param cutoff: Time limit in seconds.
    :return: (cost, tour) where cost is the total distance and tour is the list of indices.
    """
    start_time = time.time()

    # Validate input
    if not points or len(points) == 0:
        raise ValueError("No points provided for TSP")

    best_cost = float('inf')
    best_tour = []
    n = len(points)
    distance_matrix = create_distance_matrix(points)

    # Iterate through all permutations
    for perm in permutations(range(n)):
        if time.time() - start_time > cutoff:
            break
        cost = sum(distance_matrix[perm[i]][perm[i + 1]] for i in range(n - 1))
        cost += distance_matrix[perm[-1]][perm[0]]  # Complete the cycle
        if cost < best_cost:
            best_cost = cost
            best_tour = perm

    return best_cost, list(best_tour)

In [1]:
# mst_approximation.py
from scipy.sparse.csgraph import minimum_spanning_tree


def mst_approximation_tsp(points):
    """Approximates TSP using a Minimum Spanning Tree."""
    distance_matrix = create_distance_matrix(points)
    mst = minimum_spanning_tree(distance_matrix).toarray()
    visited = set()

    # Perform a preorder traversal of the MST to get the tour
    def dfs(node, tour):
        visited.add(node)
        tour.append(node)
        for neighbor in range(len(points)):
            if mst[node][neighbor] > 0 and neighbor not in visited:
                dfs(neighbor, tour)

    tour = []
    dfs(0, tour)
    tour.append(0)  # Complete the cycle

    cost = sum(distance_matrix[tour[i]][tour[i+1]] for i in range(len(tour)-1))
    return cost, tour

In [15]:
def test_mst_approximation(file_path):
    # file_path = "input/Atlanta.tsp"
    points = read_tsp(file_path)
    cost, tour = mst_approximation_tsp(points)
    print(f"Approximate TSP cost: {cost}")
    
    print(f"Approximate TSP tour: {tour}")
    return cost, tour

In [16]:
test_mst_approximation(file_path="../input/Atlanta.tsp")

Approximate TSP cost: 305974.4741402309
Approximate TSP tour: [0, 1, 2, 16, 6, 0]


(305974.4741402309, [0, 1, 2, 16, 6, 0])